# Spline curves

General properties:
- polynomial of degree $d$ for $t$
- (distance/projection equation is of degree $2d-1$)
- uniform cooficients for all segments
- locality: $d+1$ points affect a segment, $d-1$ segments affected by a point  
- parametric continuity: at least $C^0$, other depends on arrangement
- geometric continuity can be acheved by arranging control points

## General math

Piece-wise spline curve of degree $d$ with characteristic matrix $M$ of size $(d+1) × (d+1)$

$Q_i(t_l) =
\begin{bmatrix}
1 & t_l & t_l^2 & \dots & t_l^d
\end{bmatrix}
M
\begin{bmatrix}
P_i\\
P_{i+1} \\
\vdots \\
P_{i+d} \\
\end{bmatrix}
$


Each $i$-th segment runs by local $t_l \in [0, 1]$ and global $t = t_i + t_l$. The segment is generally supported by $d+1$ points $P_i, \dots, P_{i+d}$. 

Curve can extrapolate beyond the points with values of $$ outside $[0, 1]$, though. The extrapolation runs with the same speed/acceleration as at an edge point.

### Derivatives


Velocity:

$Q'(t) = \frac{d}{dt} Q(t) = 
\begin{bmatrix}
0 & 1 & 2 t & \dots & d t^{d-1}
\end{bmatrix}
M
\begin{bmatrix}\vdots \\ P \\ \vdots\end{bmatrix}
$

Acceleration:

$Q''(t) = \frac{d^2}{dt^2} Q(t) = 
\begin{bmatrix}
0 & 0 & 2 & \dots & d (d-1) t^{d-2}
\end{bmatrix}
M
\begin{bmatrix}\vdots \\ P \\ \vdots\end{bmatrix}
$

Tangent (normalized):

$T(t) = \frac{Q'(t)}{\|Q'(t)\|}$

Normal (in a plane of the osculating circle):

$N(t) = \frac{d}{dt} T(t) = \frac{Q''(t)}{\|Q''(t)\|}$

In [ ]:
import math
import random

import numpy as np
from plotly import graph_objects as go
import ipywidgets as iw
from IPython.display import display

from vectors import vec, normalize, length
import splines as spl

In [ ]:
def genpoints2d(n):
    return np.array([np.arange(n) + np.random.random(n) * 0.5, np.random.random(n)]).T

def genpoints3d(n):
    return np.array([np.arange(n) + np.random.random(n) * 0.5, np.random.random(n), np.random.random(n)]).T

In [ ]:
npoints = 6
points = genpoints2d(npoints)

pxx, pyy = points.T
go.Figure(
    [
        go.Scatter(x=pxx, y=pyy, mode="lines+markers", line=dict(dash="dot"), name="points"),
    ],
    layout=dict(yaxis=dict(range=(0, 1)), xaxis=dict(range=(0, npoints))),
)

## Bezier curves

Piece-wise curve, interpolating edge control points, approximating interior control points. 
At its edges the curve is tangent to vectors $P_1 - P_0$ and $P_{d} - P_{d-1}$.

Equivalent to non-uniform B-spline with knot vector of 0 and 1 repeated $d$ times

### Construction

Recursive:

- $Q^{(1)}_i(t|P_0, P_1) = (1-t) P_0 + (t) P_1$ — linear: lerping between 2 points
- $Q^{(2)}_i(t|P_0, P_1, P_2) = (1-t) B^{(1)}(t|P_0, P_1) + (t) B^{(1)}(t|P_1, P_2)$ — cuadratic: lerping between lerped points
- $Q^{(3)}_i(t|P_0, P_1, P_2, P_3) = (1-t) B^{(2)}(t|P_0, P_1, P_2) + (t) B^{(2)}(t|P_1, P_2, P_3)$ — cubic: lerping between lerped^2 points

Polynomial:

$b^{(d)}_k(t) = \binom{d}{k} t^k (1-t)^{(d-k)}$ — Bernstein polynomial

$Q^{(d)}(t) = \sum\limits_{k=0}^{d} b^{(d)}_k(t)P_k$

Segments supproted by generally independent $d+1$ points $P_0 \cdots P_d$, with local $t \in [0, 1]$.

### Smoothness

Continuity depends on construction and relations of the points.

- $C^0$: $P_{i,d} = P_{i+1,0}$ — shared edge point
- $G^1$ (tangent): $\|P_{i,d} - P_{i,d-1}\| = \|P_{i+1,1} - P_{i+1,0}\|$ — edge vectors are aligned with a degreee of freedom 
- $C^1$ (speed): $P_{i,d} - P_{i,d-1} = P_{i+1,1} - P_{i+1,0}$ — edge vectors are mirrored, dependant P_1 and P_{d-1} 
- $G^2, C^2$ (curvature, accel): induces fail of local control (all points dependant)

### Matrices 

$Q^{(2)}(t) =
\begin{bmatrix} 1 & t & t^2 \end{bmatrix}
\begin{bmatrix} 
1 & 0 & 0 \\ 
-2 & 2 & 0 \\ 
1 & -2 & 1 \\ 
\end{bmatrix}
\begin{bmatrix} P_0 \\ P_1 \\ P_2 \end{bmatrix}
$

$Q^{(3)}(t) =
\begin{bmatrix} 1 & t & t^2 & t^3 \end{bmatrix}
\begin{bmatrix} 
1 & 0 & 0 & 0 \\  
-3 & 3 & 0 & 0 \\
3 & -6 & 3 & 0 \\
-1 & 3 & -3 & 1 \\
\end{bmatrix}
\begin{bmatrix} P_0 \\ P_1 \\ P_2 \\ P_3 \end{bmatrix}
$

In [ ]:
npoints = 6
points = genpoints2d(npoints)

pxx, pyy = points.T

c2xx, c2yy = spl.build_curve(spl.Bezier2, points[:3])(np.linspace(0-.125, 1.125, 10)).T
c3xx, c3yy = spl.build_curve(spl.Bezier3, points[2:6])(np.linspace(0-.125, 1.125, 10)).T

go.Figure(
    [
        go.Scatter(x=pxx, y=pyy, mode="lines+markers", line=dict(dash="dot"), name="points"),
        go.Scatter(x=c2xx, y=c2yy, mode="lines", name="Bezier 2"),
        go.Scatter(x=c3xx, y=c3yy, mode="lines", name="Bezier 3"),
    ],
    layout=dict(yaxis=dict(range=(0, 1)), xaxis=dict(range=(0, npoints))),
)


## Uniform B-splines

Unlimited overlapping-piecewise curve approximating all control points.

Quadratic B-spline curve: tangent to line segments $[P_{i}, P_{i+1}]$ at midpoints

### Construction

With non-uniform knot vector $T = [t_0, t_1, ..., t_{n+d}]$ dividing curve into $n+d$ segments with any $n$ (points)

$B^{(0)}_j[T](t) =
\begin{cases}
1 \text{ if } t_{j} ≤ t < t_{j+1} \\
0 \text{ otherwise }
\end{cases}$

$B^{(d)}_j[T](t) = \frac{t - t_j}{t_{j+d} - t_j} B^{(d-1)}_j[T](t) + \frac{t_{j+d+1} - t}{t_{j+d+1} - t_{j+1}} B^{(d-1)}_{j+1}[T](t)$

With uniform knot vector $T = t_{j+1} - t_{j} = 1, t_{j} = j$

$B^{(d)}_j(t) = \frac{1}{d} ((t - j) B^{(d-1)}_j(t) + (d + 1 - (t - j)) B^{(d-1)}_{j+1}(t))$

### Smoothness

Continuos everywhere up to $C^d$ on all points.

### Matrices

With normalized $t \in [0,1]$ between points, $j = \lfloor t \rfloor$

$B^{(2)}(t) = 
\begin{bmatrix} 1 & t & t^2 \end{bmatrix}
\frac{1}{2}
\begin{bmatrix} 1 & 1 & 0 \\ -2 & 2 & 0 \\ 1 & -2 & 1 \end{bmatrix}
\begin{bmatrix} P_{j} \\ P_{j+1} \\ P_{j+2} \end{bmatrix}
$

$B^{(3)}(t) =
\begin{bmatrix} 1 & t & t^2 & t^3 \end{bmatrix}
\frac{1}{6}
\begin{bmatrix} 1 & 4 & 1 & 0 \\ -3 & 0 & 3 & 0 \\  3 & -6 & 3 & 0 \\ -1 & 3 & -3 & 1 \end{bmatrix}
\begin{bmatrix} P_{j} \\ P_{j+1} \\ P_{j+2} \\ P_{j+3} \end{bmatrix}
$



In [ ]:
npoints = 6
points = genpoints2d(npoints)

pxx, pyy = points.T
c2xx, c2yy = spl.build_curve(spl.B2, points)(np.arange(0, npoints-2, .125)).T
c3xx, c3yy = spl.build_curve(spl.B3, points)(np.arange(0, npoints-3, .125)).T

go.Figure(
    [
        go.Scatter(x=pxx, y=pyy, mode="lines+markers", line=dict(dash="dot"), name="points"),
        go.Scatter(x=c2xx, y=c2yy, mode="lines", name="B2"),
        go.Scatter(x=c3xx, y=c3yy, mode="lines", name="B3"),
    ],
    layout=dict(yaxis=dict(range=(0, 1)), xaxis=dict(range=(0, npoints))),
)

## Tangents

$\frac{d}{dt} Q(t)$ — curve velocity

normalized $\frac{d}{dt} Q(t)$ — curve tangent

In [ ]:
npoints = 6
points = genpoints2d(npoints)

M = spl.B2
d = M.shape[0]

tt = np.arange(0, npoints, .125)

curve = spl.build_curve(M, points)
tang = spl.build_curve_dt(M, points)

pxx, pyy = points.T
cxx, cyy = curve(tt).T
txx = [0, 0]
tyy = [0, 0]

fig = go.FigureWidget(
    [
        go.Scatter(x=pxx, y=pyy, mode="lines+markers", line=dict(dash="dot"), name="points", hoverinfo="skip"),
        go.Scatter(x=cxx, y=cyy, mode="lines", name="curve", line=dict(width=4), hoverinfo="none"),
        go.Scatter(x=txx, y=tyy, mode="lines+markers", name="curve/dt", marker=dict(size=10, symbol="arrow", angleref="previous"), hoverinfo="skip")
    ],
    layout=dict(yaxis=dict(range=(0, 1)), xaxis=dict(range=(0, npoints))),
)

poins_plt, curve_plt, tang_plt = fig.data

def show_tang(inds):
    if len(inds) == 0:
        tang_plt.visible = False
        return

    i = inds[0]
    tx, ty = tang(tt[i]).T
    with fig.batch_update():
        tang_plt.visible = True
        tang_plt.x = [cxx[i], cxx[i]+tx]
        tang_plt.y = [cyy[i], cyy[i]+ty]


fig.data[1].on_hover(lambda _t, pnts, _d: show_tang(pnts.point_inds))
display(fig)

# Pipe surface

Directrix (spine): 
- point: $A(u) \in R^3$ 
- tangent: $\frac{d}{du}A(u) \in R^3$

Generatrix (profile): $C(v) \in R^2$

Generatrix frame aligned to horizon:

- $z$-axis: $T(u) = A'(u)$ — spine tangent
- $x$-axis: $X(u) = T(u) \times e_z$ — horizontal and perpendicular to tangent
- $y$-axis: $Y(u) = X(u) \times T(u)$ — mutually perpendicular and approximately up

Pipe surface:

$S(u, v) = A(u) + C(v)_x X(u) + C(v)_y Y(u)$ 

### Circle profile:

$C(\phi) = r_{thicknes} \begin{bmatrix}cos(\phi) \\ sin(\phi) \end{bmatrix}$

In [ ]:
def circle(r):
    return lambda t: (r * np.cos(t), r * np.sin(t))

def build_pipe(curve, tang, profile):
    up = vec(0, 0, 1)

    @np.vectorize(signature="(),()->(3)")
    def pipe(u, v):
        a = curve(u)
        tn = normalize(tang(u))
        X = np.cross(tn, up)
        Y = np.cross(X, tn)
        x, y = profile(v)
        return a + X * x + Y * y

    return pipe

def checker(u, v):
    return np.mod(u.astype(int) + v.astype(int), 2)

In [ ]:
npoints = 6
points = genpoints3d(npoints)

M = spl.B2
d = M.shape[0]

curve = spl.build_curve(M, points)
tang = spl.build_curve_dt(M, points)
profile = circle(0.25)

pipe = build_pipe(curve, tang, profile)

tt = np.arange(0, npoints - d + 1.25, .25)
ss = np.linspace(0, math.pi, 13)
uu, vv = np.meshgrid(tt, ss)

pxx, pyy, pzz = points.T
cxx, cyy, czz = curve(tt).T
xx, yy, zz = pipe(uu, vv).T
tex = checker(uu * 4, vv * 4 / math.pi).T

go.Figure(
    [
        go.Scatter3d(x=pxx, y=pyy, z=pzz, mode="lines+markers", line=dict(dash="dot"), name="points", hoverinfo="skip"),
        go.Scatter3d(x=cxx, y=cyy, z=czz, mode="lines", line=dict(width=4), name="curve"),
        go.Surface(x=xx, y=yy, z=zz, surfacecolor=tex, hoverinfo="skip", colorscale=['black', 'gray'], showscale=False),
    ],
    layout=go.Layout(scene=dict(aspectmode="data", aspectratio={"x": 1, "y": 1, "z": 1})),
)

# Projection

Finding closest point on curve.

Distance to curve:

$\|(p - q(t^*))\|^2 = (p - q(t^*)) \cdot (p - q(t^*))$

General equation of closest point:

$(p - q(t^*)) \cdot q'(t^*) = 0$

The equation is of degree $2d-1$.

#### Problems

- generally solvable only for degrees below 3 (for linear and quadratic curves)
- multiple legit solutions possible for points inside lobe, including points with exactly the same distance  



## Linear case

$Q = M
\begin{bmatrix}
\cdots P_0 \cdots \\
\cdots P_1 \cdots \\
\end{bmatrix}
$

$q(t) = \begin{bmatrix}1 & t\end{bmatrix}\begin{bmatrix}q_0 \\ q_1\end{bmatrix} = q_0 + q_1 t$

$q'(t) = \begin{bmatrix}0 & 1\end{bmatrix}\begin{bmatrix}q_0 \\ q_1\end{bmatrix} = q_1$

$q_p = p - q_0$

Equation:

$(q_p - q_1 t) \cdot q_1 = 0$

$(q_p \cdot q_1) - (q_1 \cdot q_1) t = 0$

$\begin{bmatrix} 1 & t \end{bmatrix} \begin{bmatrix} q_p \cdot q_1 \\ - q_1 \cdot q_1 \end{bmatrix} = 0$

$t = \frac{(p - q_0) \cdot q_1}{q_1 \cdot q_1}$ 


In [406]:
def project_1(M, cpoints):
    Q = M @ cpoints

    def proj(p):
        qp = p - Q[0]
        return np.array([np.dot(qp, Q[1]) / np.dot(Q[1], Q[1])])

    return proj

## Quadratic

$
q(t) = 
\begin{bmatrix} 1 & t & t^2 \end{bmatrix}
\begin{bmatrix} q_0 \\ q_1 \\ q_2 \end{bmatrix} = 
q_0 + q_1 t + q_2 t^2 
$

$
q'(t) =
\begin{bmatrix} 0 & 1 & 2t \end{bmatrix}
\begin{bmatrix} q_0 \\ q_1 \\ q_2 \end{bmatrix} = 
q_1 + 2 q_2 t
$


$q_p = p - q_0$

Equation:

$(p - q_0 - q_1 t - q_2 t^2) \cdot (q_1 + 2 q_2 t) = 0$

$
\begin{bmatrix} 1 & t & t^2 & t^3 \end{bmatrix}
\begin{bmatrix}
(q_p \cdot q_1) \\
2 (q_p \cdot q_2) - (q_1 \cdot q_1) \\
-3 (q_1 \cdot q_2) \\
-2 (q_2 \cdot q_2) \\
\end{bmatrix}
=0
$

In [402]:
### solving by numpy for general curves

def project_2np(M, cpoints):
    Q = M @ cpoints

    def proj(p):
        qp = p - Q[0]
        poly = np.polynomial.Polynomial([
            np.dot(qp,Q[1]),
            2*np.dot(qp, Q[2]) - np.dot(Q[1], Q[1]),
            -3*np.dot(Q[1], Q[2]),
            -2*np.dot(Q[2], Q[2])
        ])
        roots = poly.roots()
        return roots[np.isreal(roots)].real

    return proj

In [403]:
def distmap(curve, tang, proj):

    @np.vectorize(signature="(),()->()")
    def dist(u, v):
        p = vec(u, v)
        tt = proj(p)
        jj = p - curve(tt)
        ll = (jj * jj).sum(1)

        lmin = np.min(ll)
        ismin = np.isclose(ll, lmin)

        t_best = tt[ismin][0]
        j_best = jj[ismin][0]
        g_best = tang(t_best)

        dist = math.sqrt(lmin)
        z_cross = j_best[0] * g_best[1] - j_best[1] * g_best[0]

        return dist if z_cross > 0 else -dist

    return dist

In [412]:
npoints = 3
points = genpoints2d(npoints)
# points = np.array([vec(0, 0), vec(1.0, 1.0), vec(2, 0)])

curve = spl.build_curve(spl.B2, points)
tang = spl.build_curve_dt(spl.B2, points)
proj = project_2np(spl.B2, points)

pxx, pyy = points.T

tt = np.linspace(0, 1, 16)
cxx, cyy = curve(tt).T

range_x = np.min(pxx), np.max(pxx)
range_y = np.min(pyy), np.max(pyy)

uu = np.linspace(*range_x, 65)
vv = np.linspace(*range_y, 65)

uuu, vvv = np.meshgrid(uu, vv)

dist = distmap(curve, tang, proj)(uuu, vvv)

txx = [0, 0]
tyy = [0, 0]

fig = go.FigureWidget(
    [
        go.Scatter(x=pxx, y=pyy, mode="lines+markers", line=dict(dash="dot"), name="points", hoverinfo="skip"),
        go.Scatter(x=cxx, y=cyy, mode="lines", name="curve", line=dict(width=4), hoverinfo="none"),
        go.Heatmap(
            x=uu,
            y=vv,
            z=dist,
            name="distmap",
            hovertemplate="x: %{x:.2f}<br>y: %{y:.2f}<br>dist: %{z:.3f}",
            colorscale="RdBu_r",
        ),
        go.Scatter(
            x=txx,
            y=tyy,
            mode="markers",
            name="proj",
            marker=dict(size=10, color="black"),
            hoverinfo="skip",
        ),
    ],
    layout=go.Layout(xaxis=dict(range=range_x, dtick=1), yaxis=dict(dtick=1, scaleanchor="x"), height=600, showlegend=False),
)

points_plt, curve_plt, dist_plt, proj_plt = fig.data


def show_proj(inds):
    if len(inds) == 0:
        proj_plt.visible = False
        return

    i, j = inds[0]
    px = uuu[i][j]
    py = vvv[i][j]
    tt = proj(vec(px, py))
    cx, cy = curve(tt).T
    with fig.batch_update():
        proj_plt.visible = True
        proj_plt.x = cx
        proj_plt.y = cy


dist_plt.on_hover(lambda _t, pnts, _d: show_proj(pnts.point_inds))

display(fig)

FigureWidget({
    'data': [{'hoverinfo': 'skip',
              'line': {'dash': 'dot'},
              'mode': 'lines+markers',
              'name': 'points',
              'type': 'scatter',
              'uid': 'dd5fe84c-20a3-4aec-88e0-b099d2e44620',
              'x': array([0.14018629, 1.38950207, 2.14711395]),
              'y': array([0.26120411, 0.95800153, 0.47681898])},
             {'hoverinfo': 'none',
              'line': {'width': 4},
              'mode': 'lines',
              'name': 'curve',
              'type': 'scatter',
              'uid': '179d40b9-1fe8-475f-b82e-c6458551d589',
              'x': array([0.76484418, 0.84703922, 0.92704892, 1.00487326, 1.08051225, 1.15396589,
                          1.22523418, 1.29431712, 1.36121471, 1.42592695, 1.48845383, 1.54879537,
                          1.60695156, 1.66292239, 1.71670787, 1.76830801]),
              'y': array([0.60960282, 0.65343825, 0.69203821, 0.7254027 , 0.75353173, 0.77642529,
                    

# Texturing

TODO